In [0]:
# Parameters
path_book = "/Volumes/workspace/dataanalysispysparkbook/bronze_files/1342-0.txt"
path_silver = "/Volumes/workspace/dataanalysispysparkbook/silver/"

In [0]:
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

book = spark.read.text(path_book)
lines = book.select(split(book.value, " ").alias("line"))
words = lines.select(explode(col("line")).alias("word"))
words_lower = words.select(lower(col("word")).alias("word_lower"))
words_clean = words_lower.select(regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word"))
words_nonull = words_clean.where(col("word") != "")
words_nonull.show(5, truncate=False)

In [0]:
results =   (words_nonull.groupby(col("word"))
                        .count().orderBy(col("count")
                        .desc())
            )
results.show(5, truncate=False)

In [0]:
import pyspark.sql.functions as F
results = (words_nonull.groupBy(F.col("word"))
                        .agg(F.count(F.col("word")).alias("count"))
                        .orderBy(F.col("count").desc())
            )
results.show(5, truncate=False)

In [0]:
results.write.csv("/Volumes/workspace/dataanalysispysparkbook/silver/simple_count.csv")

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/dataanalysispysparkbook/silver/simple_count.csv"))

In [0]:
results.coalesce(1).write.csv("/Volumes/workspace/dataanalysispysparkbook/silver/simple_count_single_partition.csv")

In [0]:
%fs
ls dbfs:/Volumes/workspace/dataanalysispysparkbook/silver/

Resumen de las instrucciones

In [0]:
from pyspark.sql.functions import (
  col,
  explode,
  lower,
  regexp_extract,
  split,
)

book = spark.read.text(path_book) # lee el archivo CSV y crear un data frame book
book.show(5, truncate=False)
lines = book.select(split(book.value, " ").alias("line")) # Separa las palabras por espacios
words = lines.select(explode(col("line")).alias("word")) # Explode las palabras en un array. Cada palabra es un elemento del array
words_lower = words.select(lower(col("word")).alias("word")) # Convierte las palabras a minusculas
words_clean = words_lower.select(regexp_extract(col("word"), "[a-z']*", 0).alias("word")) # Elimina caracteres de puntuación
words_nonull = words_clean.where(col("word") != "") # Elimina palabras vacias
results = words_nonull.groupby(col("word")).count() # Cuenta las palabras
results.orderBy("count", ascending=False).show(5) # Ordena las palabras por cantidad
(results.coalesce(1).write.format("csv") # Guarda el resultado en un archivo CSV de una sola partición
                          .mode("overwrite")
                          .save(f"{path_silver}/simple_count_single_partition.csv")
)                         

Resumiendo las instrucciones en una sola operación

In [0]:
import pyspark.sql.functions as F

results = (
  spark.read.text(path_book)
            .select(F.split(F.col("value"), " ").alias("line"))
            .select(F.explode(F.col("line")).alias("word"))
            .select(F.lower(F.col("word")).alias("word"))
            .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
            .where(F.col("word") != "")
            .groupby("word")
            .count()
            .orderBy(F.col("count").desc())
            # .coalesce(1)
            # .write.format("csv"
)
results.show(5)